# Background Information 

The slurp model requirs three essential inputs:  
1. Met.   
2. Canopy cover. 
3. Initial soil conditions. 


Model set up   
1. No notion of population or rowSpacing. but need to be provided via   
```Slurp.Sow(cultivar: StaticCrop, population: 1, depth: 10, rowSpacing: 150);```

2. Altering Slurp properties during runs  
In some cases users will wish to change properties of Slurp while the simulation is running. This can be done using a the set method in a manager script.   
```
object LAIResetValue = leaflai;
zone.Set("Slurp.Leaf.LAIFunction.Value()", LAIResetValue);
object HeightResetValue = CoverToday * MaximumHeight;
zone.Set("Slurp.Leaf.HeightFunction.Value()", HeightResetValue);
```
Model is driven by thermal time, so temperature and base temperature are critical?

A broken-stick threshold model was used in the thesis 

Tt is accumulated linearly at a rate of 0.7 °Cd/°C up to 15 °C and then at a rate of 1.0 until 30 

$T_b$ = 1
$T_t$ = 0.7 °Cd/°C when temperature below 15 
$T_t$ = 1 °Cd/°C when temperature is in 15 and 30 





# Prepare the cover data 

Light interception was not measured directly due to the instrement reported incorrect measurements for crops grown under droughts.

Frational light interception can be calculated from desctructive LAI   

$$R/R_0 = 1 - exp(-k\times LAI)$$

where (extinction coefficient) k was obtained by sunscan measurements. more details in the thesis and 2017 paper of Richard's

k values differ seasonally. 
k for all Iversen 12 and spring(Sep - Nov) and autumn(Feb - Apr) on both stone soils - **0.94 $\pm$ 0.014**  
k for summer (Dec - Jan) on both stone soils - **0.66 $\pm$ 0.013**

In [1]:
#load packages
import sqlite3
import re
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [3]:
# Build connection with db
con = sqlite3.connect('./03processed-data/Richard.sqlite3')
mycur = con.cursor() 
mycur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
(mycur.fetchall())

[('ES',),
 ('Phenology',),
 ('SoilWater',),
 ('SowingDates',),
 ('biomass',),
 ('met_AshleyDene',),
 ('met_Iversen12',),
 ('root',)]

In [4]:
# Read data in 
biomass = pd.read_sql('Select * from biomass',  con)
met_AD = pd.read_sql('Select * from met_AshleyDene', con)
met_I12 = pd.read_sql('Select * from met_Iversen12', con)

In [5]:
LAI_Height = biomass.loc[(biomass['Seed'] == 'CS')
                         & (biomass['Harvest.No.']!='Post'), 
                         ['Experiment', 'Clock.Today', 'SowingDate', 'Rep',
                          'Plot', 'Rotation.No.', 'Harvest.No.', 'Height','LAImod']]

In [10]:
# Plot that had 'Post' measurement should be out 
LAI_Height[(LAI_Height['Harvest.No.'] == 'Post') & (LAI_Height.LAImod==0)]

,Experiment,Clock.Today,SowingDate,Rep,Plot,Rotation.No.,Harvest.No.,Height,LAImod,k,Date


In [7]:
# Add the k for all 
LAI_Height['k'] = 0.94
# Replace the k for the summur crop in Ashley Dene
LAI_Height.loc[(LAI_Height['Clock.Today'] > '2011-11-30') 
               & (LAI_Height['Clock.Today'] < '2012-03-01') 
               & (LAI_Height['Experiment'] == 'AshleyDene'), 'k'] = 0.66

In [8]:
LAI_Height['Date'] = pd.to_datetime(LAI_Height['Clock.Today']).dt.strftime('%Y %b')

In [ ]:
# Check if the replace take effect
grid = sns.FacetGrid(LAI_Height, col ='Experiment', row = 'SowingDate', 
                      hue="SowingDate", palette="Set2",aspect=1.5, sharex=False)
grid.map(plt.scatter, 'Date', 'k')
grid.set_xticklabels(rotation=45)
grid.fig.tight_layout()

# Note at 20200601

The slurp seems not allow users to access [Slurp].Leaf.CoverTotal     
The setupvariable manager script is calculating LAI to feed into the model with a fixed $k$ value

In [9]:
LAI_Height

,Experiment,Clock.Today,SowingDate,Rep,Plot,Rotation.No.,Harvest.No.,Height,LAImod,k,Date
0,AshleyDene,2011-01-04,SD1,1.0,8.0,1.0,1,NaN,0.514157,0.94,2011 Jan
1,AshleyDene,2011-01-04,SD1,2.0,32.0,1.0,1,NaN,0.253992,0.94,2011 Jan
2,AshleyDene,2011-01-04,SD1,3.0,50.0,1.0,1,NaN,0.347268,0.94,2011 Jan
3,AshleyDene,2011-01-04,SD1,4.0,73.0,1.0,1,NaN,0.279894,0.94,2011 Jan
4,AshleyDene,2011-01-14,SD1,1.0,8.0,1.0,2,NaN,0.454804,0.94,2011 Jan
...,...,...,...,...,...,...,...,...,...,...,...
3119,Iversen12,2012-05-30,SD10,4.0,100.0,1.0,6,NaN,2.109737,0.94,2012 May
3120,Iversen12,2012-06-13,SD10,1.0,97.0,1.0,7,NaN,1.300146,0.94,2012 Jun
3121,Iversen12,2012-06-13,SD10,2.0,98.0,1.0,7,NaN,1.500258,0.94,2012 Jun
3122,Iversen12,2012-06-13,SD10,3.0,99.0,1.0,7,NaN,1.436579,0.94,2012 Jun


### Output LAI as the slurp input 

In [ ]:
SDs = ['SD' + str(SD) for SD in range(1, 11)]
SDs
sites = ['AshleyDene', 'Iversen12']
for site in sites: 
    for i in SDs:
        LAI_Height.loc[(LAI_Height['Experiment'] == site) & (LAI_Height.SowingDate == i),
                       ['Clock.Today', 'LAImod','k']].to_csv('./03processed-data/LAI' + site + i + '.csv',index = False)

# Calculate $R/R_0$

In [ ]:
LAI_Height['LI_frac'] = 1 - np.exp( - LAI_Height['k'] * LAI_Height['LAImod'])

In [ ]:
# Check if any weired values. e.g. over 1 or below 0 
grid = sns.FacetGrid(LAI_Height, col ='Experiment',
                      hue="SowingDate", palette="Set1",aspect=2, sharex=False)
grid.map(plt.scatter, 'Date', 'LI_frac')
grid.set_xticklabels(rotation=45)
grid.fig.tight_layout()

## Construct the index

In [ ]:
# Select only LI column
LI = LAI_Height.loc[:, ['Experiment', 'Clock.Today','SowingDate', 
                        'Rep', 'Plot', 
                        'LI_frac']]
# print_full(LI)
# remove the rows that have 0S - Likely to be wrong 0s
LI = LI[LI['LI_frac'] != 0.00]
LI

In [ ]:
LI[LI['SowingDate']=='SD5']

In [ ]:
LI = LI.pivot_table(index = 'Clock.Today', 
                    columns=['Experiment', 'SowingDate', 
                             'Rep', 'Plot'],
                    values = 'LI_frac')
LI

In [ ]:
# Check if the mean values capture all the harvest for SD5 
# SD5 has extra measurments in plot 15, 21,53,71
print_full(LI.filter(regex = 'SD5').groupby(axis = 1, level = ['Experiment','SowingDate']).mean())


In [ ]:
# Change the index to datetime tyep
LI.index = pd.to_datetime(LI.index)
# Rename the index name 
LI.index.name = 'Clock.Today'
# Normalise the datetime to midnight 
LI.index = LI.index.normalize()

In [ ]:
# Check indexs
LI.axes

## Calcuate the mean LI_frac

In [ ]:
LIGroupedMean = LI.groupby(axis=1, level=['Experiment', 'SowingDate']).mean()

In [ ]:
LIGroupedMean

In [ ]:
# Graph to check the data 

LIGroupedMean.loc[:,
                  [X for X in LIGroupedMean.columns if 'SD5' in X]].plot(figsize=(10, 6),style='o-')


In [ ]:
SDs = ['SD' + str(SD) for SD in range(1, 11)]
for i in SDs:
    LIGroupedMean.loc[:,
                  [X for X in LIGroupedMean.columns if i in X]].plot(figsize=(10, 6),style='o-')

# Met data to calculate thermal time 

In [ ]:
met_AD = met_AD.loc[:, ['year','day', 'maxt', 'mint','mean']]
met_AD['Experiment'] = 'AshleyDene'
met_I12 = met_I12.loc[(met_I12['year'] >= 2010)                       
                      & (met_I12['year'] < 2013), ['year','day', 'maxt', 'mint','mean']]
met_I12['Experiment'] = 'Iversen12'

In [ ]:
met = pd.concat([met_AD, met_I12], ignore_index=True)

In [ ]:
# Change 4 digits year to the first date of the year
met['year'] = [str(year) + '-01-01' for year in met['year']]
met['year'] = pd.to_datetime(met['year'])
# Change the day to a delta days and add back to the year 
met['Clock.Today'] = met['year'] + pd.to_timedelta(met['day'], unit='D')

In [ ]:
# Check if any weired values. 
grid = sns.FacetGrid(met, row ='Experiment',
                     aspect=2, sharex=False)
grid.map(plt.scatter, 'Clock.Today', 'mean')
grid.set_xticklabels(rotation=45)
grid.fig.tight_layout()

#### Note:   
1. The iversen 12 met has a long period of coverage.
   Expt period: **October 2010 to July 2012.**
   

In [ ]:
met = met[(met['Clock.Today'] > '2010-06-01')
          &(met['Clock.Today'] < '2012-08-01')]


In [ ]:
# Verify the filter 
# Check if any weired values. 
grid = sns.FacetGrid(met, row ='Experiment',
                     aspect=2, sharex=False)
grid.map(plt.scatter, 'Clock.Today', 'mean')
grid.set_xticklabels(rotation=45)
grid.fig.tight_layout()

In [ ]:
# indexing 
met.set_index('Clock.Today', inplace = True)

In [ ]:
print_full(met)

In [ ]:
# Try 2 sites the same time 
ThermalTimeAccum = met.loc[:, 'mean'].cumsum()
ThermalTimeAccum.index = pd.to_datetime(ThermalTimeAccum.index)

## Combine thermal time with cover data

In [ ]:
#Reindex coverdata frame to daily values
LIDaily = LI.reindex(ThermalTimeAccum.index)
LIDaily.loc[:, 'AccumTT'] = ThermalTimeAccum
# CoverDataDaily.loc[:,'AccumTT'] = ThermalTimeAccum

In [ ]:
print_full(LIDaily.filter(regex='SD5'))

## Interpolate daily values 

**Note**  
The for loop below complains _ValueError: fp and xp are not of the same length._  

This is probably because the LI is in a wide form 
But the thermal data is in a long form which includs two sites 


In [ ]:
# for p in LIDaily.columns:
#     Obs = LIDaily.loc[:,p].dropna()
#     LIDaily.loc[:,p] = np.interp(LIDaily.AccumTT, LIDaily.loc[Obs.index,'AccumTT'],Obs)

## Process sites individually   

Ashley Dene first

In [ ]:
LIAD = LI.filter(regex = 'Ashley')
#Reindex coverdata frame to daily values
TTAccumAD = met.loc[met['Experiment'] == 'AshleyDene', 'mean'].cumsum()
TTAccumAD.index = pd.to_datetime(TTAccumAD.index)

LIDailyAD = LIAD.reindex(TTAccumAD.index)
LIDailyAD.loc[:, 'AccumTT'] = TTAccumAD


In [ ]:
LIDailyAD 

In [ ]:
## Disabled because not making sense to have a starting LI 0.3 over
# for p in LIDailyAD.columns:
#     Obs = LIDailyAD.loc[:,p].dropna()
#     LIDailyAD.loc[:,p] = np.interp(LIDailyAD.AccumTT,
#                                    LIDailyAD.loc[Obs.index,'AccumTT'],Obs)

In [ ]:
LIDailyAD

In [ ]:
LIGroupedMeanAD = LIDailyAD.groupby(axis=1, level=['Experiment', 'SowingDate']).mean()

In [ ]:
LIGroupedMeanAD.columns

In [ ]:
# Graph to check the data 
LIGroupedMeanAD.loc[:,
                  [X for X in LIGroupedMeanAD.columns if X != (   'AccumTT',     '')]].plot(figsize=(10, 6),style='o-')

#### Notes

MAYBE the starting point need to be forced to be 0

## Iversen12 LI interpolate

In [ ]:
LII12 = LI.filter(regex = 'Ive')

TTAccumI12 = met.loc[met['Experiment'] == 'Iversen12', 'mean'].cumsum()
TTAccumI12.index = pd.to_datetime(TTAccumI12.index)
LIDailyI12 = LII12.reindex(TTAccumI12.index)  #Reindex coverdata frame to daily values
LIDailyI12.loc[:,'AccumTT'] = TTAccumI12
# Disabled as well 
for p in LIDailyI12.columns:
    Obs = LIDailyI12.loc[:,p].dropna()
    LIDailyI12.loc[:,p] = np.interp(LIDailyI12.AccumTT,
                                   LIDailyI12.loc[Obs.index,'AccumTT'],Obs)
LIGroupedMeanI12 = LIDailyI12.groupby(axis=1, level=['Experiment', 'SowingDate']).mean()

In [ ]:
LIGroupedMeanI12.columns

In [ ]:
# Graph to check the data 
LIGroupedMeanI12.loc[:,
                  [X for X in LIGroupedMeanI12.columns if X != (   'AccumTT',     '')]].plot(figsize=(10, 6),style='o-')

# Force the starting point to be 0

In [ ]:
sowingdates = pd.read_sql('Select * from SowingDates',  con)

In [ ]:
sowingdates.AD = pd.to_datetime(sowingdates.AD)
sowingdates.I12 = pd.to_datetime(sowingdates.I12)


In [ ]:
# set index and rename columns 
sowingdates.set_index('SD', inplace=True)
sowingdates.columns = ['AshleyDene', 'Iversen12']

In [ ]:
sowingdates.loc
LIDailyAD.axes

In [ ]:
LIAD = LI.filter(regex = 'Ashley')
#Reindex coverdata frame to daily values
TTAccumAD = met.loc[met['Experiment'] == 'AshleyDene', 'mean'].cumsum()
TTAccumAD.index = pd.to_datetime(TTAccumAD.index)

LIDailyAD = LIAD.reindex(TTAccumAD.index)
LIDailyAD.loc[:, 'AccumTT'] = TTAccumAD

In [ ]:
idx = pd.IndexSlice
LIDailyAD.loc[LIDailyAD.index == '2012-06-02', idx[:,'SD1']]

In [ ]:
for sd in sowingdates.index:
    # Select the date for correpond sowing date
    date0 = sowingdates.at[sd, 'AshleyDene']
    # A slicer
    idx = pd.IndexSlice
    # Replace the row values with 0s
    LIDailyAD.loc[LIDailyAD.index <= date0, idx[:,sd]] = float(0.001)
    # Verification 
    df = LIDailyAD.loc[LIDailyAD.index == date0, idx[:,sd]]
    print('\r')
    print(date0)     
    print(df)

#### Note   
There are still more than 4 plots for each sowing dates   
Due to seed line and harvest.no.  
The harvest.no **post** did not contribute to LAI. so should taken out.   
The Thermaltime df has duplicated the index.

In [ ]:
LIDailyAD

In [ ]:
for p in LIDailyAD.columns:
    Obs = LIDailyAD.loc[:,p].dropna()
    LIDailyAD.loc[:,p] = np.interp(LIDailyAD.AccumTT,
                                   LIDailyAD.loc[Obs.index,'AccumTT'],Obs)

In [ ]:
LIGroupedMeanADForced = LIDailyAD.groupby(axis=1, level=['Experiment', 'SowingDate']).mean()
# Graph to check the data 
LIGroupedMeanADForced.loc[:,
                          [X for X in LIGroupedMeanADForced.columns if X != (   'AccumTT',     '')]].plot(figsize=(10, 6),style='o-')

plt.savefig('./05figures/StartJune_AD.png', dpi = 300, bbox_inches = 'tight')

#### Repeat for I12


In [ ]:
LII12 = LI.filter(regex = 'Ive')

TTAccumI12 = met.loc[met['Experiment'] == 'Iversen12', 'mean'].cumsum()
TTAccumI12.index = pd.to_datetime(TTAccumI12.index)
LIDailyI12 = LII12.reindex(TTAccumI12.index)  #Reindex coverdata frame to daily values
LIDailyI12.loc[:,'AccumTT'] = TTAccumI12


In [ ]:
LIDailyI12

In [ ]:
for sd in sowingdates.index:
    # Select the date for correpond sowing date
    date0 = sowingdates.at[sd, 'Iversen12']
    # A slicer
    idx = pd.IndexSlice
    # Replace the row values with 0s
    LIDailyI12.loc[LIDailyI12.index <= date0, idx[:,sd]] = float(0.001)
    # Verification 
    df = LIDailyI12.loc[LIDailyI12.index == date0, idx[:,sd]]
    print('\r')
    print(date0)     
    print(df)

In [ ]:
# Interpolate LI daily value by thermal time 
for p in LIDailyI12.columns:
    Obs = LIDailyI12.loc[:,p].dropna()
    LIDailyI12.loc[:,p] = np.interp(LIDailyI12.AccumTT,
                                   LIDailyI12.loc[Obs.index,'AccumTT'],Obs)


In [ ]:
LIGroupedMeanI12Forced = LIDailyI12.groupby(axis=1, level=['Experiment', 'SowingDate']).mean()

In [ ]:
# Graph to check the data 
LIGroupedMeanI12Forced.loc[:,
                           [X for X in LIGroupedMeanI12Forced.columns if X != (   'AccumTT',     '')]].plot(figsize=(10, 6),style='o-')

In [ ]:
LIGroupedMeanI12Forced

In [ ]:
LIGroupedMeanADForced

# Does it make difference by changing the period of met data?

In [ ]:
sowingdates

In [ ]:
LIAD = LI.filter(regex = 'Ashley')
#Reindex coverdata frame to daily values
TTAccumAD = met.loc[(met['Experiment'] == 'AshleyDene')
                    & (met.index > '2010-10-20'), 'mean'].cumsum()
TTAccumAD.index = pd.to_datetime(TTAccumAD.index)

LIDailyAD = LIAD.reindex(TTAccumAD.index)
LIDailyAD.loc[:, 'AccumTT'] = TTAccumAD


In [ ]:
for sd in sowingdates.index:
    # Select the date for correpond sowing date
    date0 = sowingdates.at[sd, 'AshleyDene']
    # A slicer
    idx = pd.IndexSlice
    # Replace the row values with 0s
    LIDailyAD.loc[LIDailyAD.index <= date0, idx[:,sd]] = float(0.001)
    # Verification 
    df = LIDailyAD.loc[LIDailyAD.index == date0, idx[:,sd]]
for p in LIDailyAD.columns:
    Obs = LIDailyAD.loc[:,p].dropna()
    LIDailyAD.loc[:,p] = np.interp(LIDailyAD.AccumTT,
                                   LIDailyAD.loc[Obs.index,'AccumTT'],Obs)

In [ ]:
LIGroupedMeanADForced = LIDailyAD.groupby(axis=1, level=['Experiment', 'SowingDate']).mean()
# Graph to check the data 
LIGroupedMeanADForced.loc[:,
                              [X for X in LIGroupedMeanADForced.columns if X != (   'AccumTT',     '')]].plot(figsize=(10, 6),style='o-')
plt.savefig('./05figures/StartSD1_AD.png', dpi = 300, bbox_inches = 'tight')

#### Seems no difference


# Output the LI DAILY

In [ ]:
# Reset the index back to a column
LIGroupedMeanADForced.columns

In [ ]:
CoverDF = LIGroupedMeanADForced.drop('AccumTT', axis=1, level=0).stack([0,1]).reset_index()

In [ ]:
CoverDF.columns = ['Clock.Today', 'Experiment', 'SowingDate', 'SD']
CoverDF

In [ ]:
# Test
CoverDF.loc[CoverDF.SowingDate == 'SD1', ['Clock.Today', 'SD']].to_csv('./03processed-data/CoverDataSD1.csv',index = False)

In [ ]:
SDs = ['SD' + str(SD) for SD in range(1, 11)]
SDs
for i in SDs:
    CoverDF.loc[CoverDF.SowingDate == i,
                ['Clock.Today', 'SD']].to_csv('./03processed-data/CoverDataAshleyDene' + i + '.csv',index = False)

In [ ]:
# Iversen12
# The sowing dates needs to be cut off to the right one 

In [ ]:
CoverDFI12 = LIGroupedMeanI12Forced.drop('AccumTT', axis=1, level=0).stack([0,1]).reset_index()

In [ ]:
CoverDFI12.columns = ['Clock.Today', 'Experiment', 'SowingDate', 'SD']
SDs = ['SD' + str(SD) for SD in range(1, 11)]
SDs
for i in SDs:
    CoverDFI12.loc[CoverDFI12.SowingDate == i, 
                   ['Clock.Today', 'SD']].to_csv('./03processed-data/CoverDataIversen12' + i + '.csv',
                                                 index = False)